In [ ]:
from pathlib import Path
import json
import random

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image

In [ ]:
class DogsDataset(Dataset):
    def __init__(self, jsonl_path: str | Path, transform=None):
        self.jsonl_path = Path(jsonl_path)
        self.transform = transform
        with self.jsonl_path.open("r", encoding="utf-8") as f:
            self.items = [json.loads(line) for line in f if line.strip()]

        self.class_to_idx = {item["class_name"]: int(item["class_id"]) for item in self.items}
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = self.items[idx]
        image = Image.open(item["image"]).convert("RGB")
        label = int(item["class_id"])
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

training_data = DogsDataset("data/splits/dogs_train.jsonl", transform=transform)
val_data = DogsDataset("data/splits/dogs_val.jsonl", transform=transform)
test_data = DogsDataset("data/splits/dogs_test.jsonl", transform=transform)

batch_size = 32
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)

x, y = next(iter(train_dataloader))
print("train/val/test:", len(training_data), len(val_data), len(test_data))
print("x:", x.shape, "y:", y.shape, "num_classes:", len(training_data.class_to_idx))

In [ ]:
#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

in_features = x[0].numel()
num_classes = len(training_data.class_to_idx)
print("in_features:", in_features, "num_classes:", num_classes)

class NeuralNetwork(nn.Module):
    def __init__(self, in_features: int, num_classes: int):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(in_features, num_classes).to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for i, (X, y) in enumerate(dataloader):
            print(f"Processing batch {i}/{num_batches}")  # Debug print
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return 100*correct

In [ ]:
epoch = 0
epochs = 0

while True:
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    if test(test_dataloader, model, loss_fn) > 80 or (epochs != 0 and epoch+1 > epochs):
        break
    epoch+=1
print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")